In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!unrar e "/content/gdrive/My Drive/final_cleaned_data.rar" "/content"

In [0]:
!pip install spacy

In [0]:
!python -m spacy download en_core_web_lg

In [0]:
import random
import json
import numpy as np
import spacy
import nltk

In [0]:
nlp = spacy.load('en_core_web_lg')

In [0]:
def sentence_mean(nlp, s):
    if s == "":
        s = " "
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape

In [0]:
responses = {}
f = open('final_cleaned_data.json', 'r')
te = f.read() 
responses = json.loads(te)

In [0]:
sentences=[]
label=[]
for x in responses:
  sentences.append(x['message'])
  label.append(x['response'])

In [0]:
X_train_shape = (len(sentences),nlp.vocab.vectors_length)
X_train = np.zeros(X_train_shape)
# X_train

In [0]:
y_train_shape = (len(responses),nlp.vocab.vectors_length)
y_train = np.zeros(y_train_shape)
# y_train

In [0]:
for i,sentence in enumerate(sentences[:10000]):
   print(i)
   X_train[i,:] = nlp(sentence).vector

In [0]:
for i,response in enumerate(label[:10000]):
   print(i)
   y_train[i,:] = nlp(response).vector

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

In [42]:
import math
X_test_ten_thousands = X_train[:10000, :]
y_test_ten_thousands = y_train[:10000, :]

size = X_test_ten_thousands.shape[0]
per = math.floor((size * 70)/100)

X_train_s = X_test_ten_thousands[:per, :]
y_train_s = y_test_ten_thousands[:per, :]
X_test_s = X_test_ten_thousands[-(size-per):, :]
y_test_s = y_test_ten_thousands[-(size-per):, :]

print(X_train_s.shape)
print(y_train_s.shape)
print(X_test_s.shape)
print(y_test_s.shape)

# y_train_s_list = y_train_s.tolist()
# asd = np.array(y_train_s_list)
# np.reshape(asd, (7000, 1))
# print(asd.shape)
# print(len(y_train_s_list))
# print(len(y_train_s_list[0]))

(7000, 300)
(7000, 300)
(3000, 300)
(3000, 300)


In [44]:
from sklearn.svm import SVC

clf = SVC(C=1)
clf.fit(X_train_s, y_train_s)
print('Accuracy: {}'.format(clf.score(X_test_s, y_test_s)))

ValueError: ignored

In [0]:
!pip install simpleneighbors

The cell below makes a new Simple Neighbors object called `nns` and initializes it with 300 dimensions (the shape of the word vectors in spaCy, and also the shape of our summary vectors). It then samples ten thousand random conversational turns from the Cornell corpus, finds sentence vectors for each of them, and adds them to the database. (The `np.any()` line just checks to make sure that we don't add any vectors that are all zeroes by accident—this can mess up the nearest-neighbor search.)

Notes on the code below:

* I decided to just sample ten thousand turns so that the index will build faster. You can change this number to your liking!
* It only adds *turns that have responses* to the database (i.e., keys in the `responses` lookup). Because of the way the bot works, we don't need to keep track of the last turn of a conversation, since it (by definition) will have no replies.

In [0]:
from simpleneighbors import SimpleNeighbors

nns = SimpleNeighbors(300)
for i, line_id in enumerate(random.sample(list(responses.keys()), 10000)):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()

0 L147263 You're not helping.
1000 L14186 An angel, when she was having one of her headaches.
2000 L399703 Nah... scotch.
3000 L447170 That COULD BE TOLD.
4000 L327382 Why fucking not!  I deserve it.
5000 L603751 It's spectacular...
6000 L59033 Will you do something for me?
7000 L505431 That's a game isn't it?  Anyway...  There's been some interesting developments.
8000 L106937 Come on, Pop, all I want to know is one thing.  Just one thing after he made such a big deal out of it.  I bet it wasn't a big deal.  Was it, Caesar?
9000 L159864 Don't shoot, man, don't shoot!


Let's take it for a spin! The code in the following cell finds the turn most similar to the string in the variable `sentence`. (You can change this string to whatever you want.) It then uses the Simple Neighbors object to find the turn in the database with the most similar vector, and then uses the `responses` lookup to find the *response* to that turn. That response will be our bot's output.

In [0]:
sentence = "I like making bots."
picked = nns.nearest(sentence_mean(nlp, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])

Your line:
	 I like making bots.
Most similar turn:
	 I like that.
Response to most similar turn:
	 I still think we should have met them first.


## Putting it all together

The code above is all you need to make a conversational chatbot based on semantic similarity. But there's a lot of stuff to keep track of! So I wrote a little bit of "glue code" to make it even easier. You can [see the source code on GitHub](https://github.com/aparrish/semanticsimilaritychatbot/); all the important stuff is [in this file](https://github.com/aparrish/semanticsimilaritychatbot/blob/master/semanticsimilaritychatbot/__init__.py). I'm going to use this library to rewrite the code above in just a few lines, and then we'll use the resulting object to make a chatbot you can use in the browser.

First, download and install the library:

In [0]:
!pip install https://github.com/aparrish/semanticsimilaritychatbot/archive/master.zip

Then create a chatbot object, passing in the spaCy language object (`nlp`) and the number of dimensions:

In [0]:
from semanticsimilaritychatbot import SemanticSimilarityChatbot
chatbot = SemanticSimilarityChatbot(nlp, 300)

The `.add_pair()` method in the object takes two strings: a turn and the response to that turn. We'll get these from the `responses` and `movie_lines` lookups, again sampling ten thousand pairs at random. This cell will take a little while to run:

In [0]:
sample_n = 10000
for first_id, second_id in random.sample(list(responses.items()), sample_n):
    chatbot.add_pair(movie_lines[first_id], movie_lines[second_id])
chatbot.build()

Once you've built the database, the `.response_for()` method returns a plausible response from the database, based on semantic similarity. Try it out by changing the text between the quotation marks:

In [0]:
print(chatbot.response_for("Hello computer!"))

Hi there!  You alright?


To add variety, the `.response_for()` method actually selects randomly among several similar turns. You can change the number of turns it chooses from by passing a second parameter (a number) to the method. In general, the higher the number, the greater the chance is that you'll get an unusual result:

In [0]:
my_turn = "The weather's nice today, don't you think?"
for i in range(5, 51, 5):
    print("picking from", i, "possible responses:")
    print(chatbot.response_for(my_turn, i))
    print()

picking from 5 possible responses:
Well, I'd like to be lady-like and think it over.

picking from 10 possible responses:
What is it. Tammy?

picking from 15 possible responses:
Everybody does?

picking from 20 possible responses:
What sign?

picking from 25 possible responses:
I don't know. What do you feel like doing?

picking from 30 possible responses:
Have you ever considered piracy? You'd make a wonderful Dread Pirate Roberts.

picking from 35 possible responses:
Ohhh. You're in therapy too, Marty?

picking from 40 possible responses:
Yeah?

picking from 45 possible responses:
Kid stuff or not, it doesn't happen every day, I want to heat it - and if you won't say it, you can sing it...

picking from 50 possible responses:
Right. I promised my mother.



The Semantic Similarity Chatbot object has a `.save()` method that saves the pre-built database to disk, using a filename prefix you supply. (It saves three different files: `<prefix>.annoy`, `<prefix>-data.pkl`, and `<prefix>-chatbot.pkl`).

In [0]:
chatbot.save("movielines-10k-sample")

You can use a previously-saved database using the `.load()` class method, like so. (This means you don't have to build the database again: you can just load it and start calling `.response_for()`.)

In [0]:
chatbot = SemanticSimilarityChatbot.load("movielines-10k-sample", nlp)

In [0]:
print(chatbot.response_for("I'm going to go get some coffee."))

Instant rice...?


If you're using this notebook on Google Colab, the following cell will download all of the files from the pre-built bot to your computer so you can use them later. (Note that you'll still have to download and install spaCy for the chatbot to work.) If you're running the notebook locally with Jupyter Notebook, the files will end up in the same directory as the notebook file itself.

In [0]:
from google.colab import files
files.download('movielines-10k-sample.annoy')
files.download('movielines-10k-sample-data.pkl')
files.download('movielines-10k-sample-chatbot.pkl')

## Making it interactive

If you're using this notebook in Google Colab, the following cell will create a little interactive interface for chatting with the bot that you just built. Run the two cells below and start typing into the box.

In [0]:
chatbot_html = """
<style type="text/css">#log p { margin: 5px; font-family: sans-serif; }</style>
<div id="log"
     style="box-sizing: border-box;
            width: 600px;
            height: 32em;
            border: 1px grey solid;
            padding: 2px;
            overflow: scroll;">
</div>
<input type="text" id="typehere" placeholder="type here!"
       style="box-sizing: border-box;
              width: 600px;
              margin-top: 5px;">
<script>
function paraWithText(t) {
    let tn = document.createTextNode(t);
    let ptag = document.createElement('p');
    ptag.appendChild(tn);
    return ptag;
}
document.querySelector('#typehere').onchange = async function() {
    let inputField = document.querySelector('#typehere');
    let val = inputField.value;
    inputField.value = "";
    let resp = await getResp(val);
    let objDiv = document.getElementById("log");
    objDiv.appendChild(paraWithText('😀: ' + val));
    objDiv.appendChild(paraWithText('🤖: ' + resp));
    objDiv.scrollTop = objDiv.scrollHeight;
};
async function colabGetResp(val) {
    let resp = await google.colab.kernel.invokeFunction(
        'notebook.get_response', [val], {});
    return resp.data['application/json']['result'];
}
async function webGetResp(val) {
    let resp = await fetch("/response.json?sentence=" + 
        encodeURIComponent(val));
    let data = await resp.json();
    return data['result'];
}
</script>
"""

In [0]:
import IPython
from google.colab import output

display(IPython.display.HTML(chatbot_html + \
                             "<script>let getResp = colabGetResp;</script>"))

def get_response(val):
    resp = chatbot.response_for(val)
    return IPython.display.JSON({'result': resp})

output.register_callback('notebook.get_response', get_response)

If you're not using Colab, try the following two cells to install [Flask](http://flask.pocoo.org) and run a little web server from your notebook that lets you chat with the bot. Click on the link that appears below the second cell to open up the chat in a new window.

In [0]:
!pip install flask

    100% |████████████████████████████████| 92kB 4.1MB/s 
    100% |████████████████████████████████| 71kB 9.8MB/s 
    100% |████████████████████████████████| 51kB 13.9MB/s 
  Running setup.py bdist_wheel for itsdangerous ... - done
  Stored in directory: /content/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
Successfully built itsdangerous


In [0]:
from flask import Flask, request, jsonify
app = Flask(__name__)
@app.route("/response.json")
def response():
    sentence = request.args['sentence']
    return jsonify(
        {'result': chatbot.response_for(sentence)})
@app.route("/")
def home():
    return chatbot_html + "<script>let getResp = webGetResp;</script>"
app.run()

## Some things to try

If you enjoyed following along, here are some things to try:

* Use the metadata file that comes with the Cornell corpus to make a chatbot that only uses lines from a particular genre of movie. (How is a comedy chatbot different from an action chatbot?)
* Use a different corpus of conversation altogether. Your own chat logs? Conversational exchanges from a novel? Transcripts of interviews on news programs?
* Incorporate some context from the conversation when vectorizing the turns. (You might, for example, include the average of not just the given turn but also the turn that preceded it.)